In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.ui.port', '0'). \
config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [2]:
raw_df=spark.read \
    .format('csv') \
    .option('inferSchema','true') \
    .load('/public/trendytech/retail_db/order_items/part-00000')

In [20]:
product_df=spark.read \
    .format('csv') \
    .option('inferSchema','true') \
    .load('/public/trendytech/retail_db/products/part-00000')

In [21]:
product_df.show(5)

+---+---+--------------------+----+------+--------------------+
|_c0|_c1|                 _c2| _c3|   _c4|                 _c5|
+---+---+--------------------+----+------+--------------------+
|  1|  2|Quest Q64 10 FT. ...|null| 59.98|http://images.acm...|
|  2|  2|Under Armour Men'...|null|129.99|http://images.acm...|
|  3|  2|Under Armour Men'...|null| 89.99|http://images.acm...|
|  4|  2|Under Armour Men'...|null| 89.99|http://images.acm...|
|  5|  2|Riddell Youth Rev...|null|199.99|http://images.acm...|
+---+---+--------------------+----+------+--------------------+
only showing top 5 rows



In [23]:
refined_product_df=product_df.toDF('product_id','product_category_id','product_name','product_description','product_price','product_image')

In [24]:
refined_product_df.show(5)

+----------+-------------------+--------------------+-------------------+-------------+--------------------+
|product_id|product_category_id|        product_name|product_description|product_price|       product_image|
+----------+-------------------+--------------------+-------------------+-------------+--------------------+
|         1|                  2|Quest Q64 10 FT. ...|               null|        59.98|http://images.acm...|
|         2|                  2|Under Armour Men'...|               null|       129.99|http://images.acm...|
|         3|                  2|Under Armour Men'...|               null|        89.99|http://images.acm...|
|         4|                  2|Under Armour Men'...|               null|        89.99|http://images.acm...|
|         5|                  2|Riddell Youth Rev...|               null|       199.99|http://images.acm...|
+----------+-------------------+--------------------+-------------------+-------------+--------------------+
only showing top 5 

In [3]:
raw_df.show(5)

+---+---+----+---+------+------+
|_c0|_c1| _c2|_c3|   _c4|   _c5|
+---+---+----+---+------+------+
|  1|  1| 957|  1|299.98|299.98|
|  2|  2|1073|  1|199.99|199.99|
|  3|  2| 502|  5| 250.0|  50.0|
|  4|  2| 403|  1|129.99|129.99|
|  5|  4| 897|  2| 49.98| 24.99|
+---+---+----+---+------+------+
only showing top 5 rows



In [4]:
raw_df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: double (nullable = true)
 |-- _c5: double (nullable = true)



In [5]:
schema=

In [8]:
df_1=raw_df.toDF("order_item_id" , "order_id" , "product_id" , "quantity" , "subtotal","product_price")

In [9]:
df_1.show(5)

+-------------+--------+----------+--------+--------+-------------+
|order_item_id|order_id|product_id|quantity|subtotal|product_price|
+-------------+--------+----------+--------+--------+-------------+
|            1|       1|       957|       1|  299.98|       299.98|
|            2|       2|      1073|       1|  199.99|       199.99|
|            3|       2|       502|       5|   250.0|         50.0|
|            4|       2|       403|       1|  129.99|       129.99|
|            5|       4|       897|       2|   49.98|        24.99|
+-------------+--------+----------+--------+--------+-------------+
only showing top 5 rows



In [10]:
df_2=df_1.drop('subtotal')

In [12]:
df_2.show(5)

+-------------+--------+----------+--------+-------------+
|order_item_id|order_id|product_id|quantity|product_price|
+-------------+--------+----------+--------+-------------+
|            1|       1|       957|       1|       299.98|
|            2|       2|      1073|       1|       199.99|
|            3|       2|       502|       5|         50.0|
|            4|       2|       403|       1|       129.99|
|            5|       4|       897|       2|        24.99|
+-------------+--------+----------+--------+-------------+
only showing top 5 rows



In [16]:
df_2.select('order_id','product_id','quantity','product_price').show(5)

+--------+----------+--------+-------------+
|order_id|product_id|quantity|product_price|
+--------+----------+--------+-------------+
|       1|       957|       1|       299.98|
|       2|      1073|       1|       199.99|
|       2|       502|       5|         50.0|
|       2|       403|       1|       129.99|
|       4|       897|       2|        24.99|
+--------+----------+--------+-------------+
only showing top 5 rows



In [17]:
df_2.select("*","quantity*product_price as subtotal").show(5)
#wont work as it thinks the second expr as a column name

AnalysisException: cannot resolve '`quantity*product_price as subtotal`' given input columns: [order_id, order_item_id, product_id, product_price, quantity];;
'Project [order_item_id#59, order_id#60, product_id#61, quantity#62, product_price#64, 'quantity*product_price as subtotal]
+- Project [order_item_id#59, order_id#60, product_id#61, quantity#62, product_price#64]
   +- Project [_c0#16 AS order_item_id#59, _c1#17 AS order_id#60, _c2#18 AS product_id#61, _c3#19 AS quantity#62, _c4#20 AS subtotal#63, _c5#21 AS product_price#64]
      +- Relation[_c0#16,_c1#17,_c2#18,_c3#19,_c4#20,_c5#21] csv


In [18]:
from pyspark.sql.functions import expr
df_2.select("*",expr("quantity*product_price as subtotal")).show(5)

+-------------+--------+----------+--------+-------------+--------+
|order_item_id|order_id|product_id|quantity|product_price|subtotal|
+-------------+--------+----------+--------+-------------+--------+
|            1|       1|       957|       1|       299.98|  299.98|
|            2|       2|      1073|       1|       199.99|  199.99|
|            3|       2|       502|       5|         50.0|   250.0|
|            4|       2|       403|       1|       129.99|  129.99|
|            5|       4|       897|       2|        24.99|   49.98|
+-------------+--------+----------+--------+-------------+--------+
only showing top 5 rows



In [19]:
df_2.selectExpr("*","quantity*product_price as subtotal").show(5)

+-------------+--------+----------+--------+-------------+--------+
|order_item_id|order_id|product_id|quantity|product_price|subtotal|
+-------------+--------+----------+--------+-------------+--------+
|            1|       1|       957|       1|       299.98|  299.98|
|            2|       2|      1073|       1|       199.99|  199.99|
|            3|       2|       502|       5|         50.0|   250.0|
|            4|       2|       403|       1|       129.99|  129.99|
|            5|       4|       897|       2|        24.99|   49.98|
+-------------+--------+----------+--------+-------------+--------+
only showing top 5 rows



In [25]:
#increase price of each element in product
refined_product_df.withColumn('product_price','product_price*1.2').show()

AssertionError: col should be Column

In [27]:
refined_product_df.withColumn('product_price',expr('product_price*1.2')).show(5)

+----------+-------------------+--------------------+-------------------+------------------+--------------------+
|product_id|product_category_id|        product_name|product_description|     product_price|       product_image|
+----------+-------------------+--------------------+-------------------+------------------+--------------------+
|         1|                  2|Quest Q64 10 FT. ...|               null|            71.976|http://images.acm...|
|         2|                  2|Under Armour Men'...|               null|           155.988|http://images.acm...|
|         3|                  2|Under Armour Men'...|               null|107.98799999999999|http://images.acm...|
|         4|                  2|Under Armour Men'...|               null|107.98799999999999|http://images.acm...|
|         5|                  2|Riddell Youth Rev...|               null|           239.988|http://images.acm...|
+----------+-------------------+--------------------+-------------------+---------------

In [28]:
refined_product_df.withColumn('product_price',expr('CASE WHEN product_name LIKE "%Nike%" THEN 1.5*product_price WHEN product_name LIKE "%Armour%" THEN 1.25*product_price ELSE product_price')).show(20)

ParseException: 
mismatched input 'product_name' expecting {<EOF>, '-'}(line 1, pos 10)

== SQL ==
CASE WHEN product_name LIKE "%NIKE%" THEN 1.5*product_price WHEN product_name LIKE "%Armour%" THEN 1.25*product_price ELSE product_price
----------^^^
